# Draft Optimization

This notebook evaluates champion combinations using the local LoL draft prediction model to find the optimal team compositions.

In [ ]:
import pandas as pd
import requests
import itertools
from tqdm.notebook import tqdm
import os
from dotenv import load_dotenv

from utils.rl.champions import Champion

# Load API key once at the start
load_dotenv()
API_KEY = os.getenv("API_KEY")
HEADERS = {"X-API-Key": API_KEY} if API_KEY else {}

In [ ]:
patch = "15.04"
numerical_elo = 0  # highest numerical elo

## Define Champion Pools

Define the champion pools for each role. Starting with 3 champions per role.

In [ ]:
#TODO: Could also do combinations for ennemy champions, but understand that they should be limited: probably against key meta champions!

In [ ]:
# Los ratones champion pools
champion_pools = {
    "TOP": [
        Champion.JAX,
        Champion.SION,
        Champion.VOLIBEAR,
        Champion.GRAGAS,
        Champion.QUINN,
        Champion.CHOGATH,
        Champion.GAREN,
        Champion.POPPY,
        Champion.VI,
        Champion.AMBESSA,
    ],
    "JUNGLE": [
        Champion.JARVAN_IV,
        Champion.VIEGO,
        Champion.WUKONG,
        Champion.PANTHEON,
        Champion.VI,
        Champion.IVERN,
        Champion.MAOKAI,
    ],
    "MID": [
        Champion.GALIO,
        Champion.ORIANNA,
        Champion.AZIR,
        Champion.SYNDRA,
        Champion.AHRI,
        Champion.VIKTOR,
        Champion.HWEI,
        Champion.MEL,
        Champion.ZILEAN,
        Champion.CHOGATH,
    ],
    "BOT": [
        Champion.CORKI,
        Champion.EZREAL,
        Champion.JINX,
        Champion.KALISTA,
        Champion.TRISTANA,
        Champion.ZERI,
        Champion.SIVIR,
        Champion.XAYAH,
    ],
    "SUPPORT": [
        Champion.BRAUM,
        Champion.JANNA,
        Champion.RAKAN,
        Champion.RELL,
        Champion.ALISTAR,
        Champion.RENATA_GLASC,
        Champion.LULU,
        Champion.MILIO,
        Champion.BARD,
    ],
}

enemy_champion_pools = {
    "TOP": ["UNKNOWN"],
    "JUNGLE": ["UNKNOWN", Champion.MAOKAI, Champion.SKARNER],
    "MID": ["UNKNOWN"],
    "BOT": ["UNKNOWN"],
    "SUPPORT": ["UNKNOWN"],
}

# Create lookup dictionaries
id_to_name = {champion.id: champion.display_name for champion in Champion}
name_to_id = {champion.display_name: champion.id for champion in Champion}

# Display the champion pools
for role, champions in champion_pools.items():
    print(f"{role}: {', '.join([champion.display_name for champion in champions])}")

# Display the champion pools
for role, champions in champion_pools.items():
    print(
        f"ALLY {role}: {', '.join([champion.display_name for champion in champions])}"
    )

for role, champions in enemy_champion_pools.items():
    champions_display = [
        champion.display_name if isinstance(champion, Champion) else champion
        for champion in champions
    ]
    print(f"ENEMY {role}: {', '.join(champions_display)}")

## Generate Team Compositions

Generate all possible team compositions from the champion pools.

In [ ]:
def generate_team_comps(champion_pools, enemy_champion_pools=None):
    """
    Generate all possible team compositions from the champion pools,
    excluding compositions where the same champion appears in multiple roles
    across both teams.

    Args:
        champion_pools: Dictionary of champion pools for each role for ally team
        enemy_champion_pools: Dictionary of champion pools for each role for enemy team,
                             or None to use all UNKNOWNs

    Returns:
        List of team compositions, where each composition is a tuple of
        (ally_comp, enemy_comp) with each comp being a list of 5 champion objects
    """
    roles = ["TOP", "JUNGLE", "MID", "BOT", "SUPPORT"]

    # Generate all possible ally combinations
    all_ally_comps = list(itertools.product(*[champion_pools[role] for role in roles]))

    # Filter out ally compositions with duplicate champions
    valid_ally_comps = []
    for comp in all_ally_comps:
        # Create a set of champion IDs to check for uniqueness
        champion_ids = {champion.id for champion in comp if champion != "UNKNOWN"}
        # If we have the same number of unique champions as non-UNKNOWN champions, this is valid
        if len(champion_ids) == sum(1 for champion in comp if champion != "UNKNOWN"):
            valid_ally_comps.append(comp)

    # If no enemy champion pools provided, use all UNKNOWNs
    if enemy_champion_pools is None:
        # Create a single enemy comp with all UNKNOWNs
        enemy_comp = ["UNKNOWN"] * 5

        # Return combinations of valid ally comps with the single enemy comp
        return [(ally_comp, enemy_comp) for ally_comp in valid_ally_comps]

    # Generate all possible enemy combinations
    all_enemy_comps = list(
        itertools.product(*[enemy_champion_pools[role] for role in roles])
    )

    # Filter out enemy compositions with duplicate champions
    valid_enemy_comps = []
    for comp in all_enemy_comps:
        # Create a set of champion IDs to check for uniqueness
        champion_ids = {champion.id for champion in comp if champion != "UNKNOWN"}
        # If we have the same number of unique champions as non-UNKNOWN champions, this is valid
        if len(champion_ids) == sum(1 for champion in comp if champion != "UNKNOWN"):
            valid_enemy_comps.append(comp)

    # Generate all combinations of valid ally and enemy comps, excluding those with shared champions
    valid_comps = []
    for ally_comp in valid_ally_comps:
        ally_champion_ids = {
            champion.id for champion in ally_comp if champion != "UNKNOWN"
        }

        for enemy_comp in valid_enemy_comps:
            enemy_champion_ids = {
                champion.id for champion in enemy_comp if champion != "UNKNOWN"
            }

            # Check if there's no overlap between ally and enemy champions
            if not ally_champion_ids.intersection(enemy_champion_ids):
                valid_comps.append((ally_comp, enemy_comp))

    return valid_comps


# Generate all possible team compositions
team_comps = generate_team_comps(champion_pools, enemy_champion_pools)
print(f"Generated {len(team_comps)} possible team compositions")

In [ ]:
# example team comp:
team_comps[0]

## Setup Model Prediction Functions

Create functions to get predictions from the model API.

In [ ]:
def get_prediction(champion_ids, side="blue"):
    """
    Get winrate prediction for a team composition.

    Args:
        champion_ids: List of 5 champion IDs for the team
        side: 'blue' or 'red'

    Returns:
        Winrate prediction (0-1)
    """
    # Create the full 10-champion array with UNKNOWNs for the opponent team
    full_champion_ids = []

    if side == "blue":
        # Team on blue side (positions 0-4)
        full_champion_ids = champion_ids + ["UNKNOWN"] * 5
    else:
        # Team on red side (positions 5-9)
        full_champion_ids = ["UNKNOWN"] * 5 + champion_ids

    # Prepare the API request
    api_input = {
        "champion_ids": full_champion_ids,
        "numerical_elo": numerical_elo,
        "patch": patch,
    }

    # Make the API request
    try:
        response = requests.post(
            "http://0.0.0.0:8000/predict", json=api_input, headers=HEADERS
        )
        response.raise_for_status()
        if side == "blue":
            return response.json()["win_probability"]
        elif side == "red":
            return 1 - response.json()["win_probability"]
        else:
            raise ValueError("invalide side")
    except requests.exceptions.RequestException as e:
        print(f"Error making prediction: {e}")
        return None

## Optimized Batch Processing

Here's an optimized version that uses batched API calls for better performance.

In [ ]:
def evaluate_team_comps_batched():
    # Create a list to hold all API requests we need to make
    all_requests = []

    # For each composition, we need to evaluate it on both blue and red side
    for idx, (ally_comp, enemy_comp) in enumerate(team_comps):
        # Convert champions to IDs, handling UNKNOWN
        ally_ids = [champion.id for champion in ally_comp]
        enemy_ids = [
            "UNKNOWN" if champion == "UNKNOWN" else champion.id
            for champion in enemy_comp
        ]

        # Request for blue side (ally team is on blue side)
        blue_request = {
            "champion_ids": ally_ids + enemy_ids,
            "numerical_elo": numerical_elo,
            "patch": patch,
        }

        # Request for red side (ally team is on red side)
        red_request = {
            "champion_ids": enemy_ids + ally_ids,
            "numerical_elo": numerical_elo,
            "patch": patch,
        }

        # Add to our request list with metadata
        all_requests.append((blue_request, "blue", idx))
        all_requests.append((red_request, "red", idx))

    # Organize results
    blue_results = {}
    red_results = {}

    # Process in batches
    BATCH_SIZE = 16
    with tqdm(total=len(all_requests), desc="Making batch API calls") as pbar:
        for i in range(0, len(all_requests), BATCH_SIZE):
            batch = all_requests[i : i + BATCH_SIZE]

            # Extract just the API requests
            api_requests = [req[0] for req in batch]

            # Make the batch API call
            try:
                response = requests.post(
                    "http://0.0.0.0:8000/predict-batch",
                    json=api_requests,
                    headers=HEADERS,
                )
                response.raise_for_status()

                # Process results
                for (_, side, idx), prediction in zip(batch, response.json()):
                    winrate = prediction["win_probability"]

                    if side == "blue":
                        blue_results[idx] = winrate
                    else:
                        # For red side, we need to invert the probability since the API
                        # always returns probability for blue side winning
                        red_results[idx] = 1 - winrate

            except Exception as e:
                print(f"Error in batch processing: {e}")

            pbar.update(len(batch))

    # Combine results
    final_results = []
    for idx, (ally_comp, enemy_comp) in enumerate(team_comps):
        if idx in blue_results and idx in red_results:
            ally_champion_names = [champion.display_name for champion in ally_comp]
            ally_champion_ids = [champion.id for champion in ally_comp]

            enemy_champion_names = [
                "UNKNOWN" if champion == "UNKNOWN" else champion.display_name
                for champion in enemy_comp
            ]
            enemy_champion_ids = [
                "UNKNOWN" if champion == "UNKNOWN" else champion.id
                for champion in enemy_comp
            ]

            blue_winrate = blue_results[idx]
            red_winrate = red_results[idx]
            avg_winrate = (blue_winrate + red_winrate) / 2

            final_results.append(
                {
                    "ALLY_TOP": ally_champion_names[0],
                    "ALLY_JUNGLE": ally_champion_names[1],
                    "ALLY_MID": ally_champion_names[2],
                    "ALLY_BOT": ally_champion_names[3],
                    "ALLY_SUPPORT": ally_champion_names[4],
                    "ALLY_TOP_ID": ally_champion_ids[0],
                    "ALLY_JUNGLE_ID": ally_champion_ids[1],
                    "ALLY_MID_ID": ally_champion_ids[2],
                    "ALLY_BOT_ID": ally_champion_ids[3],
                    "ALLY_SUPPORT_ID": ally_champion_ids[4],
                    "ENEMY_TOP": enemy_champion_names[0],
                    "ENEMY_JUNGLE": enemy_champion_names[1],
                    "ENEMY_MID": enemy_champion_names[2],
                    "ENEMY_BOT": enemy_champion_names[3],
                    "ENEMY_SUPPORT": enemy_champion_names[4],
                    "ENEMY_TOP_ID": enemy_champion_ids[0],
                    "ENEMY_JUNGLE_ID": enemy_champion_ids[1],
                    "ENEMY_MID_ID": enemy_champion_ids[2],
                    "ENEMY_BOT_ID": enemy_champion_ids[3],
                    "ENEMY_SUPPORT_ID": enemy_champion_ids[4],
                    "blue_winrate": blue_winrate,  # Ally team on blue side
                    "red_winrate": red_winrate,  # Ally team on red side
                    "avg_winrate": avg_winrate,
                }
            )

    return final_results


# Uncomment to use the batched version instead
results = evaluate_team_comps_batched()

## Analyze Results

Create a DataFrame with the results and display the top team compositions.

In [ ]:
# Create DataFrame from results
df = pd.DataFrame(results)

# Sort by average winrate
df = df.sort_values(by='avg_winrate', ascending=False)

# Display top 10 compositions
print("Top 10 Team Compositions:")
display(df.head(10))

In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output


def create_enhanced_browser(df):
    # Create filter widgets for each role (ally team)
    ally_include_filters = {}
    ally_exclude_filters = {}

    # Create filter widgets for each role (enemy team)
    enemy_include_filters = {}
    enemy_exclude_filters = {}

    # Get roles for both teams
    ally_roles = ["ALLY_TOP", "ALLY_JUNGLE", "ALLY_MID", "ALLY_BOT", "ALLY_SUPPORT"]
    enemy_roles = [
        "ENEMY_TOP",
        "ENEMY_JUNGLE",
        "ENEMY_MID",
        "ENEMY_BOT",
        "ENEMY_SUPPORT",
    ]

    # Create filters for ally team
    for role in ally_roles:
        # Get unique champions for this role
        unique_champs = sorted(df[role].unique())

        # Create include filter
        ally_include_filters[role] = widgets.SelectMultiple(
            options=unique_champs, layout=widgets.Layout(width="180px", height="120px")
        )

        # Create exclude filter
        ally_exclude_filters[role] = widgets.SelectMultiple(
            options=unique_champs, layout=widgets.Layout(width="180px", height="120px")
        )

    # Create filters for enemy team
    for role in enemy_roles:
        # Get unique champions for this role
        unique_champs = sorted(df[role].unique())

        # Create include filter
        enemy_include_filters[role] = widgets.SelectMultiple(
            options=unique_champs, layout=widgets.Layout(width="180px", height="120px")
        )

        # Create exclude filter
        enemy_exclude_filters[role] = widgets.SelectMultiple(
            options=unique_champs, layout=widgets.Layout(width="180px", height="120px")
        )

    # Create sorting dropdown
    sort_column = widgets.Dropdown(
        options=[
            ("Average Winrate", "avg_winrate"),
            ("Blue Side Winrate", "blue_winrate"),
            ("Red Side Winrate", "red_winrate"),
        ],
        value="avg_winrate",
        description="Sort by:",
        layout=widgets.Layout(width="200px"),
    )

    # Create pagination controls
    page_size = widgets.BoundedIntText(
        value=10,
        min=1,
        max=100,
        description="Page size:",
        layout=widgets.Layout(width="150px"),
    )

    current_page = widgets.BoundedIntText(
        value=1, min=1, description="Page:", layout=widgets.Layout(width="150px")
    )

    prev_button = widgets.Button(
        description="Previous", layout=widgets.Layout(width="100px")
    )

    next_button = widgets.Button(
        description="Next", layout=widgets.Layout(width="100px")
    )

    # Create output area for displaying results
    output = widgets.Output()

    # Clear filter buttons for each role
    ally_clear_buttons = {}
    enemy_clear_buttons = {}

    for role in ally_roles:
        ally_clear_buttons[role] = widgets.Button(
            description="Clear Filters", layout=widgets.Layout(width="180px")
        )

    for role in enemy_roles:
        enemy_clear_buttons[role] = widgets.Button(
            description="Clear Filters", layout=widgets.Layout(width="180px")
        )

    # Function to apply filters and update display
    def update_display():
        with output:
            clear_output(wait=True)

            # Apply filters for each role
            filtered_df = df.copy()

            # Apply ally team filters
            for role in ally_roles:
                include_champs = list(ally_include_filters[role].value)
                exclude_champs = list(ally_exclude_filters[role].value)

                # Apply include filter (if any champions selected)
                if include_champs:
                    filtered_df = filtered_df[filtered_df[role].isin(include_champs)]

                # Apply exclude filter (if any champions selected)
                if exclude_champs:
                    filtered_df = filtered_df[~filtered_df[role].isin(exclude_champs)]

            # Apply enemy team filters
            for role in enemy_roles:
                include_champs = list(enemy_include_filters[role].value)
                exclude_champs = list(enemy_exclude_filters[role].value)

                # Apply include filter (if any champions selected)
                if include_champs:
                    filtered_df = filtered_df[filtered_df[role].isin(include_champs)]

                # Apply exclude filter (if any champions selected)
                if exclude_champs:
                    filtered_df = filtered_df[~filtered_df[role].isin(exclude_champs)]

            # Sort by the selected column
            filtered_df = filtered_df.sort_values(by=sort_column.value, ascending=False)

            # Calculate total pages
            total_rows = len(filtered_df)
            total_pages = max(1, (total_rows + page_size.value - 1) // page_size.value)

            # Update current page if needed
            if current_page.value > total_pages:
                current_page.value = total_pages

            # Calculate slice for current page
            start_idx = (current_page.value - 1) * page_size.value
            end_idx = min(start_idx + page_size.value, total_rows)

            # Display pagination info
            print(
                f"Showing {start_idx+1}-{end_idx} of {total_rows} results (Page {current_page.value} of {total_pages})"
            )

            # Create a display version of the dataframe with only the champion names and winrates
            display_columns = [
                "ALLY_TOP",
                "ALLY_JUNGLE",
                "ALLY_MID",
                "ALLY_BOT",
                "ALLY_SUPPORT",
                "ENEMY_TOP",
                "ENEMY_JUNGLE",
                "ENEMY_MID",
                "ENEMY_BOT",
                "ENEMY_SUPPORT",
                "blue_winrate",
                "red_winrate",
                "avg_winrate",
            ]

            display_df = filtered_df[display_columns].iloc[start_idx:end_idx]

            # Display the filtered and paginated data
            if total_rows > 0:
                display(display_df)
            else:
                print("No results match your filters.")

    # Connect button events
    def on_prev_click(b):
        if current_page.value > 1:
            current_page.value -= 1
            update_display()

    def on_next_click(b):
        # The max value will be calculated in update_display
        current_page.value += 1
        update_display()

    # Create clear filter handlers
    def create_ally_clear_handler(role):
        def clear_handler(b):
            ally_include_filters[role].value = ()
            ally_exclude_filters[role].value = ()
            update_display()

        return clear_handler

    def create_enemy_clear_handler(role):
        def clear_handler(b):
            enemy_include_filters[role].value = ()
            enemy_exclude_filters[role].value = ()
            update_display()

        return clear_handler

    # Connect events
    prev_button.on_click(on_prev_click)
    next_button.on_click(on_next_click)

    # Connect clear buttons
    for role in ally_clear_buttons:
        ally_clear_buttons[role].on_click(create_ally_clear_handler(role))

    for role in enemy_clear_buttons:
        enemy_clear_buttons[role].on_click(create_enemy_clear_handler(role))

    # Update when any filter or pagination control changes
    for role in ally_include_filters:
        ally_include_filters[role].observe(
            lambda change: update_display(), names="value"
        )
        ally_exclude_filters[role].observe(
            lambda change: update_display(), names="value"
        )

    for role in enemy_include_filters:
        enemy_include_filters[role].observe(
            lambda change: update_display(), names="value"
        )
        enemy_exclude_filters[role].observe(
            lambda change: update_display(), names="value"
        )

    # Update when sort column changes
    sort_column.observe(lambda change: update_display(), names="value")

    page_size.observe(lambda change: update_display(), names="value")
    current_page.observe(lambda change: update_display(), names="value")

    # Create layout for ally team filters
    ally_filter_boxes = []
    for i, role in enumerate(ally_roles):
        # Extract the role name without the "ALLY_" prefix
        role_name = role[5:]

        # Create proper headers with HTML
        include_header = widgets.HTML(value=f"<b>Include {role_name}:</b>")
        exclude_header = widgets.HTML(value=f"<b>Exclude {role_name}:</b>")

        # Layout each role's filters in a vertical box
        filter_box = widgets.VBox(
            [
                widgets.HTML(value=f"<h4>{role_name}</h4>"),
                include_header,
                ally_include_filters[role],
                exclude_header,
                ally_exclude_filters[role],
                ally_clear_buttons[role],
            ],
            layout=widgets.Layout(margin="0px", padding="0px"),
        )

        ally_filter_boxes.append(filter_box)

    # Create layout for enemy team filters
    enemy_filter_boxes = []
    for i, role in enumerate(enemy_roles):
        # Extract the role name without the "ENEMY_" prefix
        role_name = role[6:]

        # Create proper headers with HTML
        include_header = widgets.HTML(value=f"<b>Include {role_name}:</b>")
        exclude_header = widgets.HTML(value=f"<b>Exclude {role_name}:</b>")

        # Layout each role's filters in a vertical box
        filter_box = widgets.VBox(
            [
                widgets.HTML(value=f"<h4>{role_name}</h4>"),
                include_header,
                enemy_include_filters[role],
                exclude_header,
                enemy_exclude_filters[role],
                enemy_clear_buttons[role],
            ],
            layout=widgets.Layout(margin="0px", padding="0px"),
        )

        enemy_filter_boxes.append(filter_box)

    # Create horizontal layouts for ally and enemy filters
    ally_filters_row = widgets.HBox(
        ally_filter_boxes, layout=widgets.Layout(margin="0px", padding="0px")
    )
    enemy_filters_row = widgets.HBox(
        enemy_filter_boxes, layout=widgets.Layout(margin="0px", padding="0px")
    )

    # Create section headers
    ally_header = widgets.HTML(value="<h3>Ally Team Filters</h3>")
    enemy_header = widgets.HTML(value="<h3>Enemy Team Filters</h3>")

    # Create control row with sorting and pagination
    control_row = widgets.HBox(
        [sort_column, page_size, current_page, prev_button, next_button],
        layout=widgets.Layout(
            display="flex", flex_flow="row", align_items="center", width="100%"
        ),
    )

    # Main layout
    main_layout = widgets.VBox(
        [
            widgets.HTML(value="<h2>Team Composition Browser</h2>"),
            ally_header,
            ally_filters_row,
            enemy_header,
            enemy_filters_row,
            control_row,
            output,
        ]
    )

    # Initial display
    display(main_layout)
    update_display()


# Use the function with your results DataFrame
create_enhanced_browser(df)

In [ ]:
df.info()

# Export to sqlite

In [ ]:
# Export results to SQLite with champion IDs
import sqlite3

# Create SQLite database
db_path = "lol_team_compositions.db"
conn = sqlite3.connect(db_path)

# First, create a champion lookup table
conn.execute(
    """
CREATE TABLE champion_lookup (
    id INTEGER PRIMARY KEY,
    name TEXT NOT NULL
)
"""
)

# Insert all champion names into the lookup table
# Start with the special case for UNKNOWN
conn.execute("INSERT INTO champion_lookup (id, name) VALUES (0, 'UNKNOWN')")

# Insert all other champions from our lookup dictionary
for champ_id, champ_name in id_to_name.items():
    conn.execute(
        "INSERT INTO champion_lookup (id, name) VALUES (?, ?)",
        (int(champ_id), champ_name),
    )

## Lookup table for champions to filter by
# Add during your database export
conn.execute(
    """
CREATE TABLE role_champions (
    role TEXT NOT NULL,
    champion_id INTEGER NOT NULL,
    PRIMARY KEY (role, champion_id)
)
"""
)

# For each role, insert the distinct champions
roles = [
    "ally_top",
    "ally_jungle",
    "ally_mid",
    "ally_bot",
    "ally_support",
    "enemy_top",
    "enemy_jungle",
    "enemy_mid",
    "enemy_bot",
    "enemy_support",
]

for role in roles:
    # Get distinct champions for this role
    distinct_champs = df[f"{role.upper()}_ID"].unique()

    # Insert each champion for this role
    for champ_id in distinct_champs:
        if champ_id == "UNKNOWN":
            champ_id = 0
        else:
            champ_id = int(champ_id)

        conn.execute(
            "INSERT INTO role_champions (role, champion_id) VALUES (?, ?)",
            (role, champ_id),
        )

# Create the main team compositions table with numeric IDs
conn.execute(
    """
CREATE TABLE team_comps (
    id INTEGER PRIMARY KEY,
    ally_top_id INTEGER NOT NULL,
    ally_jungle_id INTEGER NOT NULL,
    ally_mid_id INTEGER NOT NULL,
    ally_bot_id INTEGER NOT NULL,
    ally_support_id INTEGER NOT NULL,
    enemy_top_id INTEGER NOT NULL,
    enemy_jungle_id INTEGER NOT NULL,
    enemy_mid_id INTEGER NOT NULL,
    enemy_bot_id INTEGER NOT NULL,
    enemy_support_id INTEGER NOT NULL,
    blue_winrate REAL NOT NULL,
    red_winrate REAL NOT NULL,
    avg_winrate REAL NOT NULL
)
"""
)

# Create indices for faster filtering
conn.execute("CREATE INDEX idx_ally_top_id ON team_comps(ally_top_id)")
conn.execute("CREATE INDEX idx_ally_jungle_id ON team_comps(ally_jungle_id)")
conn.execute("CREATE INDEX idx_ally_mid_id ON team_comps(ally_mid_id)")
conn.execute("CREATE INDEX idx_ally_bot_id ON team_comps(ally_bot_id)")
conn.execute("CREATE INDEX idx_ally_support_id ON team_comps(ally_support_id)")
conn.execute("CREATE INDEX idx_enemy_top_id ON team_comps(enemy_top_id)")
conn.execute("CREATE INDEX idx_enemy_jungle_id ON team_comps(enemy_jungle_id)")
conn.execute("CREATE INDEX idx_enemy_mid_id ON team_comps(enemy_mid_id)")
conn.execute("CREATE INDEX idx_enemy_bot_id ON team_comps(enemy_bot_id)")
conn.execute("CREATE INDEX idx_enemy_support_id ON team_comps(enemy_support_id)")
conn.execute("CREATE INDEX idx_avg_winrate ON team_comps(avg_winrate)")
conn.execute("CREATE INDEX idx_blue_winrate ON team_comps(blue_winrate)")
conn.execute("CREATE INDEX idx_red_winrate ON team_comps(red_winrate)")

# Prepare data for insertion
# We need to convert the DataFrame to use numeric IDs consistently
# and handle the UNKNOWN case by setting it to 0
data_to_insert = []
for _, row in df.iterrows():
    # Convert enemy champion IDs from "UNKNOWN" strings to 0
    enemy_top_id = 0 if row["ENEMY_TOP_ID"] == "UNKNOWN" else int(row["ENEMY_TOP_ID"])
    enemy_jungle_id = (
        0 if row["ENEMY_JUNGLE_ID"] == "UNKNOWN" else int(row["ENEMY_JUNGLE_ID"])
    )
    enemy_mid_id = 0 if row["ENEMY_MID_ID"] == "UNKNOWN" else int(row["ENEMY_MID_ID"])
    enemy_bot_id = 0 if row["ENEMY_BOT_ID"] == "UNKNOWN" else int(row["ENEMY_BOT_ID"])
    enemy_support_id = (
        0 if row["ENEMY_SUPPORT_ID"] == "UNKNOWN" else int(row["ENEMY_SUPPORT_ID"])
    )

    # Create a tuple with all the data for this row
    data_tuple = (
        int(row["ALLY_TOP_ID"]),
        int(row["ALLY_JUNGLE_ID"]),
        int(row["ALLY_MID_ID"]),
        int(row["ALLY_BOT_ID"]),
        int(row["ALLY_SUPPORT_ID"]),
        enemy_top_id,
        enemy_jungle_id,
        enemy_mid_id,
        enemy_bot_id,
        enemy_support_id,
        row["blue_winrate"],
        row["red_winrate"],
        row["avg_winrate"],
    )
    data_to_insert.append(data_tuple)

# Insert data in batches for better performance
BATCH_SIZE = 1000
for i in range(0, len(data_to_insert), BATCH_SIZE):
    batch = data_to_insert[i : i + BATCH_SIZE]
    conn.executemany(
        """
    INSERT INTO team_comps (
        ally_top_id, ally_jungle_id, ally_mid_id, ally_bot_id, ally_support_id,
        enemy_top_id, enemy_jungle_id, enemy_mid_id, enemy_bot_id, enemy_support_id,
        blue_winrate, red_winrate, avg_winrate
    ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    """,
        batch,
    )

# Commit and close
conn.commit()
conn.close()

print(f"SQLite database created at: {db_path}")
print(
    f"- Created champion_lookup table with {len(id_to_name) + 1} entries (including UNKNOWN)"
)
print(f"- Created team_comps table with {len(df)} entries")
print(f"- All champion IDs are stored as integers (with UNKNOWN as 0)")
print(f"- Frontend can use champion_lookup table to convert IDs to names as needed")